# CrewAI: Orchestrating Collaborative Agent Systems

**Objective:** This notebook provides a hands-on guide to CrewAI, a framework designed for engineering sophisticated multi-agent systems. You will learn how to create a "crew" of autonomous AI agents that collaborate to achieve complex goals.

**Target Audience:** Software engineers attending the AI-Driven Software Engineering Program.

**Core Philosophy:** CrewAI's strength lies in its intuitive, high-level abstractions for defining agents with specific roles, assigning them tasks, and defining the workflow (process) they should follow. It simplifies the creation of systems where multiple agents work together, each contributing its specialized skills.

## 1. Setup

First, we'll install the necessary libraries. CrewAI uses LangChain components under the hood. We will also install the `tavily-python` library directly to create a custom search tool, which helps avoid complex dependency issues.

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

if not os.getenv("OPENAI_API_KEY") or not os.getenv("TAVILY_API_KEY"):
    print("ERROR: OPENAI_API_KEY or TAVILY_API_KEY not found. Please check your .env file.")

## 2. Foundational Crew: The Two-Agent Team

Our first example will be a simple, two-agent crew designed to plan a trip. This will introduce the three core concepts of CrewAI:

-   **`Agent`**: A role-playing entity with a specific `role`, `goal`, and `backstory`.
-   **`Task`**: A specific unit of work to be performed by an agent.
-   **`Crew`**: A collection of agents and tasks, along with a defined `process` for execution.

In [2]:
from crewai import Agent, Task, Crew, Process
from langchain_openai import ChatOpenAI

# Use a LangChain model as the LLM for the agents
llm = ChatOpenAI(model="gpt-4o")

# Agent 1: The Travel Expert
travel_agent = Agent(
    role='Expert Travel Agent',
    goal='Create a detailed, 3-day itinerary for a trip.',
    backstory='You have 20 years of experience in luxury travel planning and know all the hidden gems.',
    verbose=True,
    llm=llm
)

# Agent 2: The Local Food Critic
food_critic = Agent(
    role='Local Food Critic',
    goal='Recommend the best, most authentic local restaurants for the trip.',
    backstory='You are a famous food blogger who lives in the destination city and is an expert on its culinary scene.',
    verbose=True,
    llm=llm
)

# Task 1: Plan the Itinerary (for the Travel Agent)
plan_itinerary = Task(
    description='Create a 3-day travel itinerary for a trip to Tokyo, Japan. Focus on cultural sites and activities.',
    expected_output='A markdown file with a day-by-day plan, including morning, afternoon, and evening activities.',
    agent=travel_agent
)

# Task 2: Recommend Restaurants (for the Food Critic)
# This task uses the output of the first task as its context.
recommend_restaurants = Task(
    description='Based on the planned itinerary, recommend one authentic restaurant for each day of the trip.',
    expected_output='A list of 3 restaurant names, each with a brief description and why it fits the itinerary.',
    agent=food_critic,
    context=[plan_itinerary]
)

# Form the crew with a sequential process
trip_crew = Crew(
    agents=[travel_agent, food_critic],
    tasks=[plan_itinerary, recommend_restaurants],
    process=Process.sequential,
    verbose=True
)

print("--- Kicking off the Trip Planning Crew ---")
trip_result = trip_crew.kickoff()

print("\n--- Final Itinerary ---")
print(trip_result)

--- Kicking off the Trip Planning Crew ---

--- Final Itinerary ---
### Day 1: Introduction to Tokyo's Traditional Culture
- **Nabezo Asakusa Kaminarimon**: Located just a few minutes from Senso-ji Temple, this restaurant offers an authentic kaiseki meal, perfect for lunch. The restaurant is well-reviewed for its elegant presentation and dedication to traditional Japanese flavors, aligning with the day's exploration of traditional culture in Asakusa.

### Day 2: Modern Meets Traditional
- **Izakaya Den**: Situated near Omotesando Avenue, Izakaya Den provides a casual yet immersive dining experience with a variety of small plates and local sake. The relaxed atmosphere and array of dishes allow you to enjoy a laid-back lunch amidst your modern and traditional cultural explorations in Harajuku and Omotesando.

### Day 3: Discovering Historical and Cultural Depths
- **Sushi Daiwa**: Although the Tsukiji Fish Market has moved to Toyosu, Sushi Daiwa remains a renowned spot near the old marke

## 3. Adding Tools to a CrewAI Agent

CrewAI agents can use tools to access external information and capabilities. Instead of relying on pre-built wrappers which can cause dependency issues, we can create our own custom tool by wrapping the Tavily Python client. This is a robust and flexible approach.

In [3]:
from crewai.tools import BaseTool
from tavily import TavilyClient

# 1. Create a custom tool by inheriting from BaseTool
class TavilySearchTool(BaseTool):
    name: str = "TavilySearch"
    description: str = "A tool that can be used to search the web with Tavily for up-to-date information."
    
    def _run(self, query: str) -> str:
        client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))
        response = client.search(query=query, search_depth="basic")
        return response['results']

# Instantiate our custom tool
search_tool = TavilySearchTool()

# 2. Create an agent and assign the custom tool to it
market_researcher = Agent(
    role='Market Research Analyst',
    goal='Find and summarize the latest news about a specific company.',
    backstory='You are a skilled analyst who can quickly find and synthesize financial news.',
    tools=[search_tool], # Assign the custom tool here
    verbose=True,
    llm=llm
)

# 3. Create a task for the agent
research_company_task = Task(
    description='Find the top 3 news headlines for NVIDIA since September 2, 2025, summarize them and provide the news date',
    expected_output='A numbered list of 3 headlines, each followed by a one-sentence summary.',
    agent=market_researcher
)

# 4. Form a single-agent crew to run the task
research_crew = Crew(
    agents=[market_researcher],
    tasks=[research_company_task],
    verbose=True
)

print("--- Kicking off the Research Crew with Tools ---")
research_result = research_crew.kickoff()

print("\n--- Final Research Summary ---")
print(research_result)

--- Kicking off the Research Crew with Tools ---

--- Final Research Summary ---
1. NVIDIA First Company in History at $5 Trillion Value After Major Announcements at GTC 2025 (Dated: Sep 18th, 2025): NVIDIA's valuation reached $5 trillion following significant announcements at the GTC 2025, although a $500 billion GPU sales figure was clarified as a misinterpretation.

2. NVIDIA Announces Financial Results for Second Quarter Fiscal 2026 (Dated: This was published in reference to fiscal results without a direct new date): NVIDIA released its financial results for the second quarter of fiscal 2026, detailing reconciliations that adjust GAAP financial measures for various expenses.

3. Nvidia becomes first public company worth $5 trillion (Dated: Oct 29, 2025): Nvidia has surpassed a $5 trillion market cap as the first public company to do so, largely due to its role as a significant player in the AI boom.


## 4. Advanced Capability: Hierarchical Process

For more complex workflows, CrewAI offers a `hierarchical` process. In this mode, a manager agent is dynamically nominated to orchestrate the crew, delegating tasks to other agents and synthesizing the final result. This is suitable for problems that require more dynamic coordination.

In [4]:
# Ensure Task, Crew, and Process are available (in case earlier cells haven't been run)
from crewai import Task, Crew, Process

# Re-using the agents from our first example

# The tasks are the same, but we don't need to specify context
plan_itinerary_h = Task(
    description='Create a 3-day travel itinerary for a trip to Rome, Italy. Focus on historical sites.',
    expected_output='A markdown file with a day-by-day plan.',
    agent=travel_agent
)

recommend_restaurants_h = Task(
    description='Recommend one authentic restaurant for each day of the trip to Rome.',
    expected_output='A list of 3 restaurant names with descriptions.',
    agent=food_critic
)

# Form the crew with a hierarchical process
hierarchical_crew = Crew(
    agents=[travel_agent, food_critic],
    tasks=[plan_itinerary_h, recommend_restaurants_h],
    process=Process.hierarchical,  # Use the hierarchical process
    manager_llm=llm # Specify an LLM for the manager agent
)

print("--- Kicking off the Hierarchical Crew ---")
hierarchical_result = hierarchical_crew.kickoff()

print("\n--- Final Hierarchical Result ---")
print(hierarchical_result)

--- Kicking off the Hierarchical Crew ---

--- Final Hierarchical Result ---
**Day 1: Ancient Rome Essentials**

- **Lunch: Trattoria Luzzi**  
  Trattoria Luzzi is a classic Roman trattoria offering an authentic homestyle dining experience. Located near the Colosseum, it serves traditional Roman dishes like Cacio e Pepe and Amatriciana, providing a genuine taste of Roman cuisine. The bustling atmosphere, popular with locals, makes it a perfect introduction to Roman dining.

- **Dinner: Ristorante Aroma**  
  Overlooking the Colosseum, Ristorante Aroma provides a refined dining experience with authentic Italian flavors presented with a modern twist. Known for high-quality ingredients and traditional techniques, it offers a luxurious yet genuine Italian meal, making it a memorable dining spot to cap off a day exploring Rome's history.

**Day 2: Vatican City & Baroque Rome**

- **Lunch: Osteria Delle Commari**  
  This charming restaurant near the Vatican focuses on traditional Roman cui

## Lab Conclusion

In this lab, you've learned the fundamentals of CrewAI. You've seen how to define agents with distinct roles, create tasks for them to perform, and assemble them into a crew that can work together sequentially or be managed hierarchically.

**Key Takeaways:**
- CrewAI is excellent for problems that can be broken down into distinct roles and responsibilities.
- The `Agent`, `Task`, and `Crew` abstractions provide a clear and intuitive way to structure multi-agent systems.
- Creating custom tools by inheriting from `BaseTool` is a reliable way to add capabilities and avoid dependency conflicts.
- The `hierarchical` process allows for more dynamic, manager-led coordination for complex, non-linear tasks.